# Great Weapon Fighting
These are some math examples of the values you get for the Great Weapon Fighting style in 5e D&D
which let you reroll 1 or 2 on any weapon damage dice.

## Doing Some Math

In [16]:
import numpy as np
import pandas as pd

# Pulling this model from AceCalhoon's answer
# https://rpg.stackexchange.com/questions/47172/how-much-damage-does-great-weapon-fighting-add-on-average

series = {}
for i in range(4, 13, 2):
    series[i] = 1 - (2/i)
df = pd.DataFrame(series.items(), columns=['Die Type', 'Difference'])
df

ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

## Code to generate the values

In [5]:
def gwf_reroll(roll, sides):
    """
    This checks the value of roll and rolls again if a 2 or 1 and returns it.
    :param roll: int of roll value
    :param sides: int of die side to use.
    :return: int of new roll value
    """
    if roll < 3:
        return np.random.randint(1, sides + 1)
    return roll

def hr_add_reroll(roll, sides):
    """
    This checks the value of roll and adds a new roll if it's under a 3.
    :param roll: int of roll value
    :param sides: int of die side to use
    :return: int of new roll value
    """
    if roll < 3:
        new_roll = roll + np.random.randint(1, sides + 1)
        return new_roll
    return roll

def hr_higher_reroll(roll, sides):
    """
    This keeps the higher of either the original roll or reroll.
    :param roll: int of roll value
    :param sides: int of die side to use
    :return: int of new roll value
    """
    roll_2 = 0
    if roll < 3:
        roll_2 = np.random.randint(1, sides + 1)
    return max(roll, roll_2)

def damage_roll(sides, num_die=1, gwf=False, crit_on=20, reroll_fn=gwf_reroll):
    """
    This rolls a number of die and returns the result, rerolling 1s or 2s if gwf is true.
    :param sides: int sided die to use
    :param num_die: int number of die
    :param gwf: bool to indicate using gwf rerolls
    :param crit_on: int of value or above on d20 to become a crit
    :param reroll_fn: function to use for any reroll
    :return: int of roll total
    """
    # Crits double the number of rolls
    is_crit = np.random.randint(1, 21) >= crit_on
    num_actual = num_die * 2 if is_crit else num_die

    result = 0
    for _ in range(num_actual):
        roll = np.random.randint(1, sides + 1) # high is exlusive
        if gwf:
            roll = reroll_fn(roll, sides)
        result = roll + result

    return result

## Running the simulation for various die types.

In [6]:
# Setup a means to generate the full data.
def generate_weapon_data(gwf=False, reroll_fn=gwf_reroll):
    crit_range = {'Base': 100, '20': 20, '19': 19}
    die_range = {
        '1d8': (8, 1),
        '1d10': (10, 1),
        '1d12': (12, 1),
        '2d6': (6, 2)}
    weapon_data = {}
    for cl, cv in crit_range.items():
        weapon_data[cl] = {}
        for dk, dv in die_range.items():
            weapon_data[cl][dk] = pd.Series(damage_roll(dv[0], dv[1], gwf=gwf, crit_on=cv, reroll_fn=reroll_fn) for _ in range(10000)).mean()
    return weapon_data

### Normal (No GWF)

In [7]:
normal_df = pd.DataFrame(generate_weapon_data())
normal_df.round(1)

,Base,20,19
1d8,4.5,4.7,4.9
1d10,5.5,5.8,6.0
1d12,6.5,6.8,7.1
2d6,7.0,7.4,7.7


### With GWF

In [8]:
gwf_df = pd.DataFrame(generate_weapon_data(gwf=True))
gwf_df.round(1)

,Base,20,19
1d8,5.2,5.5,5.8
1d10,6.3,6.6,7.0
1d12,7.4,7.7,8.0
2d6,8.4,8.7,9.2


## Various House Rules

### Add new roll to previous roll option.

In [9]:
hr_add_df = pd.DataFrame(generate_weapon_data(gwf=True, reroll_fn=hr_add_reroll))
hr_add_df.round(1)

,Base,20,19
1d8,5.6,5.9,6.2
1d10,6.6,6.9,7.3
1d12,7.6,8.0,8.4
2d6,9.3,9.8,10.3


## Keep the higher of the rolls

In [10]:
hr_higher_df = pd.DataFrame(generate_weapon_data(gwf=True, reroll_fn=hr_higher_reroll))
hr_higher_df.round(1)

,Base,20,19
1d8,5.3,5.5,5.8
1d10,6.3,6.6,6.9
1d12,7.3,7.7,8.1
2d6,8.4,8.8,9.3
